In [4]:
## TEMP for development only before moving into .py files

import os
import pandas as pd
import formula_page_generator
import utilities
import topics


website_creator_dir = os.getcwd()
docs_dir = utilities.get_docs_path(website_creator_dir)
formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
syllabus_file_path = (website_creator_dir + os.path.sep +
                        'syllabus_topics.csv')
sort_orders_df = pd.read_csv(order_file_path)
syllabus_df = pd.read_csv(syllabus_file_path)
formulas_input_df = pd.read_csv(formula_file_path)
formula_sheet_items = (
    formula_page_generator.get_formulas_on_formula_sheet(formulas_input_df))
formula_proof_required_items = (
    formula_page_generator.get_formulas_where_proofs_required(formulas_input_df))

utilities.delete_directory_if_it_exists(docs_dir)

# Create formula-related markdown documents
formulas_df = formula_page_generator.get_formulas_df(
    formulas_input_df, syllabus_df, sort_orders_df)
formula_page_generator.create_formulas_content(
    formulas_df, formula_sheet_items, formula_proof_required_items,
    sort_orders_df, docs_dir)
formula_page_generator.create_calculus_summary_files(
    formulas_df, formula_sheet_items, formula_proof_required_items,
    sort_orders_df, docs_dir)
formula_page_generator.create_financial_summary(
    formulas_df, formula_sheet_items, formula_proof_required_items,
    sort_orders_df, docs_dir)

# Generate topic pages
formulas_by_topic_df = formula_page_generator.get_formulas_by_topic_df(
    formulas_input_df, syllabus_df)

topics.create_topic_content(formulas_by_topic_df, formula_sheet_items,
                            formula_proof_required_items, sort_orders_df,
                            docs_dir)


In [1]:
import os
import utilities
from maths_objects import Formulas, WebPageHierarchies

website_creator_dir = os.getcwd()
docs_dir = utilities.get_docs_path(website_creator_dir)
formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
syllabus_file_path = (website_creator_dir + os.path.sep +
                        'syllabus_topics.csv')

page_sort_orders = WebPageHierarchies(order_file_path)
school_formulas = Formulas(formula_file_path, syllabus_file_path,
                           page_sort_orders)



In [7]:
def create_index_files(base_dir, dirs_df, front_matter=None,
                       sort_orders_df=None):
    """Creates _index.md files recursively at each level of the directory
    strucutres formed by joining base_dir string and each row of pandas
    dataframe dirs_df.  The content of each file is set based  on the content
    of the front_matter dictionary and optionally includes a weight
    (sort order) based on the order of each directory in sort_orders_df """
    if front_matter is None:
        front_matter = {}
    for i in range(1, len(dirs_df.columns)+1):
        # Looping through various levels in directory structure
        subdirs_df = dirs_df.iloc[:, :i].drop_duplicates()
        for _, row in subdirs_df.iterrows():
            file_name = (base_dir + os.path.sep +
                         os.path.sep.join(row) + os.path.sep + '_index.md')
            if not os.path.isfile(file_name):
                # Don't overwrite file if it exists
                # Use a copy on each loop to start fresh and not
                # carry over any details from previos loop
                front_matter_to_write = front_matter.copy()
                if sort_orders_df is not None:
                    sort_order = lookup_list_in_df(
                        sort_orders_df, list(row.values))
                    if sort_order is not None:
                        front_matter_to_write['weight'] = sort_order + 1
                string_to_write = get_front_matter_string(
                    front_matter_to_write)
                with open(file_name, "w", encoding="utf-8") as text_file:
                    text_file.write(string_to_write)

1